# First Model Rundown

## Set Up the Environment

In [ ]:
#  Mounts Google Colab on Gdrive.
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Selects our Gdrive we just mounted above
%cd /content/gdrive/My Drive

# Create our working directory
%mkdir HuruAI

# Move into the working directory
%cd /HuruAI

In [ ]:
# The code below sets us up with some nice formatting for our plots.

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Import the required packages

import numpy as np

import torch
from torchvision import datasets, transforms

from torch import nn, optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import plot

## Preparing the Dataset

In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
# Download and transform the training dataset
trainset = datasets.MNIST('./Data', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and transform the test dataset
testset = datasets.MNIST('./Data', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [ ]:
train_iterator = iter(trainloader)
images, labels = train_iterator.next()
print(type(images))
print(images.shape)
print(labels.shape)

## Building the Model

In [ ]:
# Defining Our Network Architecture.

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        
        return x

## Training the Model

In [ ]:
# Instantiate the Model, define the loss and optimizer

model = Net()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [ ]:
# Flatten the into a 1-D vector, new shape is (batch size, color channels, total image pixels) 

images.resize_(64, 1, 784)

# alternatively use the code below to automatically load the batch size
# images.resize_(images.shape[0], 1, 784)

# Make a forward pass through the network
image_index = 0
probs = model.forward(images[image_index,:])

image = images[image_index]
plot.image_preds(image.view(1, 28, 28), probs)

In [ ]:
# Training the Network

epochs = 5

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        logits = model(images)
        loss = criterion(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

## Testing the Model

In [ ]:
# Create an iterator for the test dataset
test_iterator = iter(testloader)
images, labels = test_iterator.next()
image = images[1]

# Test the network 
probs = torch.exp(model(image))

# Plot the image and class probabilities
plot.image_preds(image, probs)